# Collect census info

1. collect ACS census tract information for Cook County.
2. collect whole US data at the census tract level.

Need to install censusdata module from Github.
https://jtleider.github.io/censusdata/


In [1]:
import pandas as pd
import censusdata

In [11]:
# please personalize the data directories
data_dir = '../data/'
raw_data_dir = '/Users/shenhaowang/Dropbox (MIT)/DOE_TSMS/00_Data/raw_data/'

In [2]:
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.precision', 2)

In [3]:
# Check the ACS lookup table here:
# https://www.census.gov/programs-surveys/acs/technical-documentation/summary-file-documentation.2015.html
# Targeting data: income, population, age, gender, and race
# Relevant tables: B00001, B00002, B01001, B01002, B02001, B06011
# censusdata.search('acs5', 2015, 'label', 'Block Group')
# censusdata.search('acs5', 2015, 'label', 'Census Tract')
# censusdata.search('acs5', 2015, 'label', 'Block')


In [4]:
# check vars here.
censusdata.printtable(censusdata.censustable('acs5', 2015, 'B06010'))

Variable     | Table                          | Label                                                    | Type 
-------------------------------------------------------------------------------------------------------------------
B06010_001E  | B06010. Place of Birth by Indi | Total:                                                   | int  
B06010_002E  | B06010. Place of Birth by Indi | No income                                                | int  
B06010_003E  | B06010. Place of Birth by Indi | With income:                                             | int  
B06010_004E  | B06010. Place of Birth by Indi | !! With income: $1 to $9,999 or loss                     | int  
B06010_005E  | B06010. Place of Birth by Indi | !! With income: $10,000 to $14,999                       | int  
B06010_006E  | B06010. Place of Birth by Indi | !! With income: $15,000 to $24,999                       | int  
B06010_007E  | B06010. Place of Birth by Indi | !! With income: $25,000 to $34,999           

In [5]:
# variable list
var_list = ['B00001_001E', 'B00002_001E', 
            'B01001_001E', 'B01001_002E', 'B01001_026E', 
            'B01002_001E', 
            'B02001_001E', 'B02001_002E', 'B02001_003E', 'B02001_004E', 'B02001_005E', 
            'B06010_001E', 'B06010_002E', 'B06010_003E', 'B06010_004E', 'B06010_005E', 'B06010_006E', 'B06010_007E', 'B06010_008E', 'B06010_009E', 'B06010_010E', 'B06010_011E']

var_names = ['total_pop', 'total_houses', 
             'total_sex', 'male', 'female',
             'median_age',
             'total_race', 'white', 'black', 'native', 'asian',
             'total_pop_inc', 'pop_no_inc', 'pop_with_inc', 'pop_inc_10k', 'pop_inc_1k_15k', 'pop_inc_15k_25k', 'pop_inc_25k_35k', 'pop_inc_35k_50k', 'pop_inc_50k_65k', 'pop_inc_65k_75k', 'pop_inc_75k']


In [6]:
# cook data
cook_tract = censusdata.download('acs5', 2015,
                                 censusdata.censusgeo([('state', '17'), ('county', '031'), ('tract', '*')]),
                                 var_list)

In [7]:
print(cook_tract.shape)

(1319, 22)


In [8]:
# replace names
cook_tract.columns = var_names

# clean cook county data
state_fips=['17']*cook_tract.shape[0]
county_fips=['031']*cook_tract.shape[0]
tract_fips=[]
full_tract_fips=[]

for i in range(cook_tract.shape[0]):
    tract_fips.append(cook_tract.index[i].params()[2][1])
    full_tract_fips.append('17'+'031'+cook_tract.index[i].params()[2][1])

cook_tract['state_fips']=state_fips
cook_tract['county_fips']=county_fips
cook_tract['tract_fips']=tract_fips
cook_tract['full_tract_fips']=full_tract_fips

# reset index
cook_tract.reset_index(drop = True, inplace = True)
cook_tract

,total_pop,total_houses,total_sex,male,female,median_age,total_race,white,black,native,...,pop_inc_15k_25k,pop_inc_25k_35k,pop_inc_35k_50k,pop_inc_50k_65k,pop_inc_65k_75k,pop_inc_75k,state_fips,county_fips,tract_fips,full_tract_fips
0,134,69,1185,615,570,34.3,1185,43,1117,0,...,58,97,42,66,24,45,17,031,271400,17031271400
1,329,221,6811,3355,3456,31.5,6811,5538,285,19,...,463,159,519,840,765,3058,17,031,280100,17031280100
2,309,128,4643,2307,2336,37.0,4643,2638,577,92,...,694,366,533,252,66,279,17,031,823500,17031823500
3,290,164,1858,800,1058,29.4,1858,64,1766,0,...,193,175,106,101,29,54,17,031,821500,17031821500
4,432,160,5369,2514,2855,36.2,5369,3930,1077,0,...,370,287,666,614,148,662,17,031,821800,17031821800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1314,361,192,3426,1896,1530,28.3,3426,2892,47,0,...,100,179,401,478,270,910,17,031,070300,17031070300
1315,308,146,6558,3549,3009,34.5,6558,3960,1773,0,...,955,657,646,544,142,550,17,031,010600,17031010600
1316,383,163,4518,2288,2230,38.6,4518,2733,688,0,...,416,371,350,290,163,557,17,031,020400,17031020400
1317,351,181,3801,1960,1841,35.9,3801,2933,198,0,...,455,373,516,544,186,698,17,031,031000,17031031000


In [9]:
cook_tract.shape

(1319, 26)

In [12]:
# state number list
state_df = pd.read_csv(raw_data_dir+'census/state_fips.csv')
state_dir = {}
for key,item in zip(state_df['Postal Code'], state_df['FIPS']):
    # first, control the digits
    item = str(item)
    if len(str(item)) == 1:
        item = '0'+str(item)
    # 
    if str(item) not in ['60','66','69','72','78','02','15']:
        state_dir[item] = key # ahhh...the order sucks
        
state_dir
# state_dic = []

{'01': 'AL',
 '04': 'AZ',
 '05': 'AR',
 '06': 'CA',
 '08': 'CO',
 '09': 'CT',
 '10': 'DE',
 '12': 'FL',
 '13': 'GA',
 '16': 'ID',
 '17': 'IL',
 '18': 'IN',
 '19': 'IA',
 '20': 'KS',
 '21': 'KY',
 '22': 'LA',
 '23': 'ME',
 '24': 'MD',
 '25': 'MA',
 '26': 'MI',
 '27': 'MN',
 '28': 'MS',
 '29': 'MO',
 '30': 'MT',
 '31': 'NE',
 '32': 'NV',
 '33': 'NH',
 '34': 'NJ',
 '35': 'NM',
 '36': 'NY',
 '37': 'NC',
 '38': 'ND',
 '39': 'OH',
 '40': 'OK',
 '41': 'OR',
 '42': 'PA',
 '44': 'RI',
 '45': 'SC',
 '46': 'SD',
 '47': 'TN',
 '48': 'TX',
 '49': 'UT',
 '50': 'VT',
 '51': 'VA',
 '53': 'WA',
 '54': 'WV',
 '55': 'WI',
 '56': 'WY'}

In [13]:
# us tract data
us_tract = {}

for key in state_dir.keys():
    print(state_dir[key])
    us_tract[key] = censusdata.download('acs5', 2015,
                                 censusdata.censusgeo([('state', key), ('tract', '*')]),
                                 var_list)

AL
AZ
AR
CA
CO
CT
DE
FL
GA
ID
IL
IN
IA
KS
KY
LA
ME
MD
MA
MI
MN
MS
MO
MT
NE
NV
NH
NJ
NM
NY
NC
ND
OH
OK
OR
PA
RI
SC
SD
TN
TX
UT
VT
VA
WA
WV
WI
WY


In [14]:
us_tract_df = pd.concat(us_tract.values())

In [15]:
# replace names
us_tract_df.columns = var_names

# clean cook county data
state_fips=[]
county_fips=[]
tract_fips=[]
full_tract_fips=[]

for i in range(us_tract_df.shape[0]):
    state_fips.append(us_tract_df.index[i].params()[0][1])
    county_fips.append(us_tract_df.index[i].params()[1][1])
    tract_fips.append(us_tract_df.index[i].params()[2][1])
    full_tract_fips.append(us_tract_df.index[i].params()[0][1]+us_tract_df.index[i].params()[1][1]+us_tract_df.index[i].params()[2][1])

us_tract_df['state_fips']=state_fips
us_tract_df['county_fips']=county_fips
us_tract_df['tract_fips']=tract_fips
us_tract_df['full_tract_fips']=full_tract_fips

# reset index
us_tract_df.reset_index(drop = True, inplace = True)
us_tract_df

,total_pop,total_houses,total_sex,male,female,median_age,total_race,white,black,native,...,pop_inc_15k_25k,pop_inc_25k_35k,pop_inc_35k_50k,pop_inc_50k_65k,pop_inc_65k_75k,pop_inc_75k,state_fips,county_fips,tract_fips,full_tract_fips
0,219.0,112.0,4792,2037,2755,29.6,4792,3078,1358,145,...,538,670,264,111,107,158,01,103,005109,01103005109
1,225.0,127.0,5739,2347,3392,43.3,5739,4648,799,0,...,918,477,420,345,80,436,01,103,005106,01103005106
2,337.0,140.0,4883,2700,2183,43.5,4883,4502,106,0,...,445,651,489,290,106,523,01,103,005107,01103005107
3,284.0,134.0,3787,1811,1976,45.8,3787,2912,684,15,...,495,394,351,138,80,580,01,103,005108,01103005108
4,214.0,97.0,2784,1418,1366,38.4,2784,2716,0,39,...,230,164,197,113,107,182,01,103,005701,01103005701
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72354,588.0,308.0,4406,2113,2293,50.6,4406,3971,15,62,...,608,508,546,410,102,444,56,013,000300,56013000300
72355,258.0,182.0,1714,855,859,56.3,1714,1615,23,24,...,341,203,233,145,84,100,56,013,000400,56013000400
72356,478.0,168.0,5871,2853,3018,38.1,5871,4934,8,595,...,752,631,559,421,198,467,56,013,940400,56013940400
72357,341.0,148.0,3710,1697,2013,37.1,3710,3485,0,36,...,441,212,320,330,128,324,56,045,951300,56045951300


In [16]:
print(us_tract_df.shape)

(72359, 26)


## Save files

In [17]:
import pickle

with open(data_dir+'cook_tract.pickle', 'wb') as f:
    pickle.dump(cook_tract, f)

with open(data_dir+'usa_tract.pickle', 'wb') as f:
    pickle.dump(us_tract_df, f)
    
    